## 独自のコードインタープリターを構築する - o3-miniによる動的ツール生成と実行

LLMエージェントが外部世界や他のエージェントと相互作用する機能の中核となるのは「ツール（または関数）呼び出し」です。これは、LLMが引数を持つ関数（コードブロック）を呼び出すことができる機能です。通常、これらの関数は開発者によって事前に定義され、期待される入力と出力も決められています。しかし、このクックブックでは、より柔軟なパラダイムを探求します。LLMモデル（この場合は**o3-mini**）を使用して**動的にツールを生成**し、コードインタープリターを使ってそのツールを実行する能力について説明します。

### コードインタープリターによる動的生成ツール呼び出し
動的生成ツールとは、ユーザーのプロンプトに基づいてLLM自体が実行時に作成する関数またはコードブロックです。これは、コードベースで考えられるすべてのシナリオを事前に定義する必要がないことを意味し、はるかにオープンエンドで創造的、かつ適応的な問題解決を可能にします。

動的生成ツール呼び出しは、LLMにツールを生成し、コードブロックをその場で実行する能力を与えることで、さらに一歩進んだアプローチです。この動的アプローチは、以下のようなタスクに特に有用です：

- データ分析と可視化
- データ操作と変換
- 機械学習ワークフローの生成と実行
- プロセス自動化とスクリプト作成
- その他、実験を通じて新たな可能性が生まれる多くの分野

### 動的ツール生成におけるo3-miniの使用

2025年1月31日にリリースされたo3-miniモデルは、科学、数学、コーディングに特に優れた例外的なSTEM能力を持ちながら、小型モデルの低コストと短いレイテンシを維持しています。このクックブックでは、データを解釈し洞察を導き出すPythonコードを生成するo3-miniの能力を実証します。

推論モデルは、明示的な思考の連鎖プロンプトを必要とせず、自分自身で推論できるため、データを分析する動的ツールの生成に特に優れています。実際、明示的な思考の連鎖指示を提供すると、モデルの内部推論に干渉し、最適でない結果につながる可能性があります。o3-miniについて詳しくは[こちら](https://openai.com/index/openai-o3-mini/)をご覧ください。

### 独自のコードインタープリターを構築する理由

OpenAIのAssistants APIなど、多くのAPIプロバイダーは組み込みのコードインタープリター機能を提供しています。これらの組み込みコードインタープリターは非常に強力ですが、開発者が独自のカスタムコードインタープリターを作成する必要がある状況があります。例えば：

1. **言語やライブラリのサポート**: 組み込みインタープリターが、タスクに必要な特定のプログラミング言語（C++、Javaなど）やライブラリをサポートしていない場合。
2. **タスクの互換性**: ユースケースがプロバイダーの組み込みソリューションと互換性がない場合。
3. **モデルの制約**: プロバイダーのインタープリターでサポートされていない言語モデルが必要な場合。
4. **コストの考慮**: コード実行やモデル使用のコスト構造が予算や制約に合わない場合。
5. **ファイルサイズ**: 入力データのファイルサイズが大きすぎる、またはプロバイダーのインタープリターでサポートされていない場合。
6. **内部システムとの統合**: プロバイダーのインタープリターが内部システムと統合できない場合。

### 学習内容
このクックブックに従うことで、以下のことを学習できます：

- Dockerを使用した分離されたPythonコード実行環境のセットアップ
- LLMエージェント用の独自コードインタープリターツールの設定
- セキュリティと安全性のための「エージェント的」関心事の明確な分離の確立
- データ分析用のコードを動的に生成するための**o3-mini**モデルの使用
- 与えられたタスクを効率的に達成するためのエージェントのオーケストレーション
- コードを動的に生成・実行できるエージェントアプリケーションの設計

ゼロからカスタムコードインタープリターツールを構築し、LLMの力を活用して洗練されたコードを生成し、そのコードを分離された環境で安全に実行する方法を学びます。これらすべては、AI駆動アプリケーションをより柔軟で強力、かつコスト効率的にすることを目的としています。

### サンプルシナリオ

[Key Factors Traffic Accidents](https://www.kaggle.com/datasets/willianoliveiragibin/key-factors-traffic-accidents)で提供されているサンプルデータを使用して、一連の質問に答えます。これらの質問は事前に定義する必要はなく、LLMにそのような質問に答えるためのコードを生成する能力を与えます。

サンプル質問は以下のようなものです：
- 事故頻度に最も寄与する要因は何か？（特徴量重要度分析）
- 事故のリスクが最も高い地域はどこか？（分類/クラスタリング）
- 交通違反金額は事故件数にどのような影響を与えるか？（回帰/因果推論）
- 事故率を減らすための最適な罰金額を決定できるか？（最適化モデル）
- より高い罰金は平均速度の低下や事故の減少と相関があるか？（相関/回帰）
- その他...

従来の**事前定義ツール呼び出し**アプローチを使用する場合、開発者はこれらの質問それぞれに対して関数を事前に定義する必要があります。これにより、事前定義された関数セットで定義されていない他の質問に答えるLLMの能力が制限されます。私たちは、LLMがコードを生成し、Code Interpretterツールを使用してコードを実行する**動的ツール呼び出し**アプローチを使用することで、この制限を克服します。

## 概要
動的に生成されるツール呼び出し機能を持つエージェントアプリケーションを構築する手順について詳しく見ていきましょう。このアプリケーションには3つのコンポーネントがあります：

#### ステップ1: 分離されたコード実行コンテナ環境のセットアップ

LLMが生成した関数呼び出しを実行できる安全な環境が必要です。LLMが生成したコードをホストマシン上で直接実行することを避けたいので、リソースアクセスが制限された（例：ネットワークアクセスなし）Dockerコンテナ環境を作成します。デフォルトでは、Dockerコンテナはホストマシンのファイルシステムにアクセスできないため、LLMが生成したコードが確実に隔離された状態を保つことができます。

##### ⚠️ 注意事項: LLMが生成するコードに対する強力なガードレールの実装
LLMは意図しない結果を招く有害なコードを生成する可能性があります。ベストプラクティスとして、タスクに必要なリソースへのアクセスのみを許可し、コード実行環境を分離してください。LLMが生成したコードをホストマシンやラップトップ上で実行することは避けてください。

#### ステップ2: エージェントの定義とテスト

「**エージェントとは何か？**」このCookbookの文脈において、エージェントとは以下のものです：
1. LLMが従うべき指示のセット、つまり開発者プロンプト
2. LLMモデル、およびAPI経由でモデルを呼び出す機能
3. 関数へのツール呼び出しアクセス、および関数を実行する機能

2つのエージェントを定義します：
1. FileAccessAgent: このエージェントはファイルを読み取り、PythonCodeExecAgentにコンテキストを提供します。
2. PythonCodeExecAgent: このエージェントはユーザーの質問に答えるPythonコードを生成し、Dockerコンテナ内でコードを実行します。

#### ステップ3: アプリケーションを実行するためのエージェント オーケストレーションのセットアップ
アプリケーションの要件に基づいて、エージェントをオーケストレーションする様々な方法があります。この例では、ユーザーがタスクを提供し、エージェントが順次呼び出されてタスクを達成するシンプルなオーケストレーションを使用します。

全体的なオーケストレーションを以下に示します：

![](../../images/oo_aa_image_1_code_interpreter_agents.png)

## 始めましょう


### 前提条件
開始する前に、ホストマシンに以下がインストールされ、設定されていることを確認してください：

1. Docker: ローカルマシンにインストールされ、実行されていること。Dockerについて詳しく学び、[こちらからインストール](https://www.docker.com/)できます。
2. Python: ローカルマシンにインストールされていること。Pythonについて詳しく学び、[こちらからインストール](https://www.python.org/downloads/)できます。
3. OpenAI API key: ローカルマシンで環境変数として、またはルートディレクトリの.envファイルに設定されていること。OpenAI API keyについて詳しく学び、[こちらから設定](https://platform.openai.com/docs/api-reference/introduction)できます。

### ステップ1: 分離されたコード実行環境のセットアップ

コードを実行するために使用するDockerizedコンテナ環境を定義しましょう。`resources/docker`ディレクトリの下に**[dockerfile](https://github.com/openai/openai-cookbook/blob/main/examples/object_oriented_agentic_approach/resources/docker/dockerfile)**を定義しており、以下の仕様でコンテナ環境を作成するために使用されます：
- ベースとしてPython 3.10を使用
- 非rootユーザー
- requirements.txtのパッケージを事前インストール

dockerイメージ作成プロセスに含まれるrequirements.txtには、LLMが生成するコードがタスクを実行するために必要となる可能性のあるすべてのパッケージが含まれています。コンテナからのネットワークアクセスを制限するため、タスクに必要なパッケージを事前にインストールする必要があります。セキュリティ上の理由から、LLMは追加のパッケージをインストールすることは許可されません。

言語要件（Python 3.10など）を満たし、タスクに必要なパッケージを事前インストールした独自のdockerイメージを作成することも、特定の言語（Java、C++など）とタスクに必要なパッケージを含むカスタムdockerイメージを作成することも可能です。

以下のコマンドでDockerイメージをビルドしましょう。簡潔にするため、出力を成功メッセージをgrepするようにリダイレクトし、ビルドが失敗した場合にメッセージを出力するようにしています。

In [1]:
!docker build -t python_sandbox:latest ./resources/docker 2>&1 | grep -E "View build details|ERROR" || echo "Build failed."

View build details: docker-desktop://dashboard/build/desktop-linux/desktop-linux/kl8fo02q7rgbindi9b42pn1zr


制限モードでコンテナを実行してみましょう。コンテナはバックグラウンドで実行されます。これは、コンテナのセキュリティポリシーを定義する機会です。タスクに必要な最小限の機能のみをコンテナに許可することが良い慣行です。デフォルトでは、コンテナはコンテナ内からホストファイルシステムにアクセスできません。また、インターネットやその他のネットワークリソースにアクセスできないように、ネットワークへのアクセスも制限しましょう。

In [2]:
# Run the container in restricted mode. The container will run in the background.
!docker run -d --name sandbox --network none --cap-drop all --pids-limit 64 --tmpfs /tmp:rw,size=64M   python_sandbox:latest sleep infinity

8446d1e9a7972f2e00a5d1799451c1979d34a2962aa6b4c35a9868af8d321b0e


`docker ps`を使用してコンテナが実行されていることを確認しましょう。これにより、私たちのコンテナが一覧表示されるはずです。

In [3]:
!docker ps 

CONTAINER ID   IMAGE                   COMMAND            CREATED         STATUS         PORTS     NAMES
8446d1e9a797   python_sandbox:latest   "sleep infinity"   2 seconds ago   Up 2 seconds             sandbox


### ステップ2: エージェントの定義とテスト

我々の目的のために、2つのエージェントを定義します。

1.	**エージェント1: ファイルアクセスエージェント（事前定義されたツール呼び出し）**
- エージェント2にコンテキストとして提供するファイルの内容を理解する指示。
- ホストマシンのファイルシステムにアクセス可能。
- ホストからファイルを読み取り、Dockerコンテナにコピー可能。
- コードインタープリターツールにはアクセス不可。
- gpt-4oモデルを使用。

2.	**エージェント2: Pythonコード生成・実行エージェント（動的に生成されたツール呼び出しとコード実行）**
- エージェント1からファイルコンテンツのコンテキストを受信。
- ユーザーの質問に答えるPythonスクリプトを生成する指示。
- Dockerコンテナ内のコードインタープリターにアクセス可能で、Pythonコードの実行に使用。
- Dockerコンテナ内のファイルシステムのみにアクセス可能（ホストではない）。
- ホストマシンのファイルシステムやネットワークにはアクセス不可。
- コード生成に優れた最新の**o3-mini**モデルを使用。

ファイルアクセス（エージェント1）とコード生成・実行（エージェント2）の関心事の分離は、LLMがホストマシンに直接アクセスしたり変更したりすることを防ぐために重要です。

**エージェント1はホストファイルシステムにアクセスできるため、静的ツール呼び出しに制限します。**

| エージェント | ツール呼び出しの種類 | ホストファイルシステムへのアクセス | Dockerコンテナファイルシステムへのアクセス | コードインタープリターへのアクセス |
|-------|-------------------|----------------------------|----------------------------------------|----------------------------|
| エージェント1: ファイルアクセス | 事前定義されたツール | あり | あり | なし |
| エージェント2: Pythonコード生成・実行 | 動的に生成されたツール | なし | あり | あり |

エージェントとツールを整理するために、オブジェクト指向プログラミングの原則を使用して一貫性を保ちながら2つのエージェントを作成するための**コアクラス**のセットを定義しました。

- **BaseAgent**: `task()`などの共通メソッドシグネチャを強制する抽象基底クラスから始めます。基底クラスはデバッグ用のロガー、言語モデルインターフェース、およびエージェントにコンテキストを追加する`add_context()`などの他の共通機能も提供します。
- **ChatMessages**: ChatCompletions APIがステートレスであることを考慮して、会話履歴を保存するクラスです。
- **ToolManager**: エージェントが呼び出すことができるツールを管理するクラスです。
- **ToolInterface**: エージェントが呼び出すことができる任意の「ツール」のための抽象クラスで、ツールが一貫したインターフェースを持つようにします。

これらのクラスは[object_oriented_agents/core_classes](./resources/object_oriented_agents/core_classes)ディレクトリに定義されています。

#### コアクラスのUMLクラス図
以下のクラス図は、コアクラス間の関係を示しています。このUML（統一モデリング言語）は[Mermaid](https://mermaid)を使用して生成されています。

![](../../images/oo_aa_image_2_uml_diagram.png)

**エージェント1の定義：FileAccessToolを持つFileAccessAgent**

まず、ToolInterfaceクラスを継承するFileAccessToolの定義から始めましょう。**FileAccessTool**ツールは、`resources/registry/tools`ディレクトリの[file_access_tool.py](https://github.com/openai/openai-cookbook/blob/main/examples/object_oriented_agentic_approach/resources/registry/tools/file_access_tool.py)ファイルで定義されています。

- FileAccessToolはToolInterfaceクラスを実装しており、これによりツールが一貫したインターフェースを持つことが保証されます。
- `get_definition`メソッドでOpenAI Function Calling APIのツール定義をバインドし、ツールの`run`メソッドと組み合わせることで、保守性、拡張性、再利用性が確保されます。

それでは、BaseAgentクラスを継承し、**FileAccessTool**をエージェントにバインドする**FileAccessAgent**を定義しましょう。FileAccessAgentは、`resources/registry/agents`ディレクトリの[file_acess_agent.py](https://github.com/openai/openai-cookbook/blob/main/examples/object_oriented_agentic_approach/resources/registry/agents/file_access_agent.py)ファイルで定義されています。FileAccessAgentは以下の特徴を持ちます：

- BaseAgentクラスの具象実装です。
- 開発者プロンプト、モデル名、ロガー、言語モデルインターフェースで初期化されます。これらの値は必要に応じて開発者がオーバーライドできます。
- FileAccessToolをツールマネージャーに登録する`setup_tools`メソッドを持ちます。
- ファイルを読み取り、PythonCodeExecAgentにコンテキストを提供するためにFileAccessToolを呼び出す`task`メソッドを持ちます。
- タスクに十分な推論とツール呼び出し能力を提供する`model_name='gpt-4o'`を使用します。

**エージェント2の定義: PythonExecToolを使用したPythonExecAgent**

同様に、PythonExecToolはToolInterfaceクラスを継承し、get_definitionメソッドとrunメソッドを実装しています。get_definitionメソッドは、OpenAI Function Calling APIが期待する形式でツール定義を返します。runメソッドは、DockerコンテナでPythonコードを実行し、出力を返します。このツールは、`resources/registry/tools`ディレクトリの[python_code_interpreter_tool.py](https://github.com/openai/openai-cookbook/blob/main/examples/object_oriented_agentic_approach/resources/registry/tools/python_code_interpreter_tool.py)ファイルで定義されています。

同様に、PythonExecAgentはBaseAgentクラスの具体的な実装です。これは、`resources/registry/agents`ディレクトリの[python_code_exec_agent.py](https://github.com/openai/openai-cookbook/blob/main/examples/object_oriented_agentic_approach/resources/registry/agents/python_code_exec_agent.py)ファイルで定義されています。PythonExecAgentは以下の特徴があります：

- BaseAgentクラスの具体的な実装
- 開発者プロンプト、モデル名、ロガー、言語モデルインターフェースで初期化される。これらの値は必要に応じて開発者が上書きできる
- PythonExecToolをツールマネージャーに登録するsetup_toolsメソッドを持つ
- ユーザーのタスクを実行するためにOpenAI APIを呼び出す`task`メソッドを持つ。この場合、ユーザーの質問に答えるPythonスクリプトを生成し、Code Interpreterツールで実行する
- コード生成などのSTEMタスクに優れた`model_name='o3-mini'`
- タスクの複雑さに応じてより完全な推論を可能にする`reasoning_effort='high'`。ただし、より多くのトークンが生成され、応答が遅くなるコストがある。デフォルト値はmediumで、速度と推論精度のバランスを取っている

`reasoning_effort`パラメータについて詳しくは[こちら](https://platform.openai.com/docs/guides/reasoning)をご覧ください。

### ステップ3: アプリケーションを実行するためのエージェント・オーケストレーションを設定する

エージェントが定義されたので、次にアプリケーションを実行するオーケストレーション・ループを定義できます。このループはユーザーに質問やタスクを入力するよう促し、その後FileAccessAgentを呼び出してファイルを読み取り、PythonExecAgentにコンテキストを提供します。PythonExecAgentはユーザーの質問に答えるためのPythonコードを生成し、Dockerコンテナ内でコードを実行します。コード実行の出力がユーザーに表示されます。

ユーザーは'exit'と入力してアプリケーションを停止できます。私たちの質問は：**事故頻度に最も寄与する要因は何ですか？** この質問に答える関数を事前に定義していないことに注意してください。

In [4]:
# Import the agents from registry/agents

from resources.registry.agents.file_access_agent import FileAccessAgent
from resources.registry.agents.python_code_exec_agent import PythonExecAgent


prompt = """Use the file traffic_accidents.csv for your analysis. The column names are:
Variable	Description
accidents	Number of recorded accidents, as a positive integer.
traffic_fine_amount	Traffic fine amount, expressed in thousands of USD.
traffic_density	Traffic density index, scale from 0 (low) to 10 (high).
traffic_lights	Proportion of traffic lights in the area (0 to 1).
pavement_quality	Pavement quality, scale from 0 (very poor) to 5 (excellent).
urban_area	Urban area (1) or rural area (0), as an integer.
average_speed	Average speed of vehicles in km/h.
rain_intensity	Rain intensity, scale from 0 (no rain) to 3 (heavy rain).
vehicle_count	Estimated number of vehicles, in thousands, as an integer.
time_of_day	Time of day in 24-hour format (0 to 24).
accidents	traffic_fine_amount
"""


print("Setup: ")
print(prompt)

print("Setting up the agents... ")

# Instantiate the agents with the default constructor defined values
# Developer may override the default values - prompt, model, logger, and language model interface if needed

# This agent use gpt-4o by default
file_ingestion_agent = FileAccessAgent()

# Let's make sure agent uses o3-mini model and set the reasoning_effort to high
data_analysis_agent = PythonExecAgent(model_name='o3-mini', reasoning_effort='high')

print("Understanding the contents of the file...")
# Give a task to the file ingestion agent to read the file and provide the context to the data analysis agent 
file_ingestion_agent_output = file_ingestion_agent.task(prompt)

# Add the file content as context to the data analysis agent
# The context is added to the agent's tool manager so that the tool manager can use the context to generate the code 

data_analysis_agent.add_context(prompt)
data_analysis_agent.add_context(file_ingestion_agent_output)

while True:

    print("Type your question related to the data in the file. Type 'exit' to exit.")
    user_input = input("Type your question.")

    if user_input == "exit":
        print("Exiting the application.")
        break

    print(f"User question: {user_input}")

    print("Generating dynamic tools and using code interpreter...")
    data_analysis_agent_output = data_analysis_agent.task(user_input)

    print("Output...")
    print(data_analysis_agent_output)


Setup: 
Use the file traffic_accidents.csv for your analysis. The column names are:
Variable	Description
accidents	Number of recorded accidents, as a positive integer.
traffic_fine_amount	Traffic fine amount, expressed in thousands of USD.
traffic_density	Traffic density index, scale from 0 (low) to 10 (high).
traffic_lights	Proportion of traffic lights in the area (0 to 1).
pavement_quality	Pavement quality, scale from 0 (very poor) to 5 (excellent).
urban_area	Urban area (1) or rural area (0), as an integer.
average_speed	Average speed of vehicles in km/h.
rain_intensity	Rain intensity, scale from 0 (no rain) to 3 (heavy rain).
vehicle_count	Estimated number of vehicles, in thousands, as an integer.
time_of_day	Time of day in 24-hour format (0 to 24).
accidents	traffic_fine_amount

Setting up the agents... 
Understanding the contents of the file...


2025-02-03 13:03:54,066 - MyApp - INFO - Handling tool call: safe_file_access
2025-02-03 13:03:54,067 - MyApp - INFO - Tool arguments: {'filename': './resources/data/traffic_accidents.csv'}
2025-02-03 13:03:54,562 - MyApp - INFO - Tool 'safe_file_access' response: Copied ./resources/data/traffic_accidents.csv into sandbox:/home/sandboxuser/.
The file content for the first 15 rows is:
    accidents  traffic_fine_amount  traffic_density  traffic_lights  pavement_quality  urban_area  average_speed  rain_intensity  vehicle_count  time_of_day
0          20               4.3709           2.3049         753.000            0.7700           1        321.592          1.1944       290.8570     160.4320
1          11               9.5564           3.2757           5.452            4.0540           1        478.623          6.2960       931.8120       8.9108
2          19               7.5879           2.0989           6.697          345.0000           0        364.476          2.8584       830.086

Type your question related to the data in the file. Type 'exit' to exit.
User question: What factors contribute the most to accident frequency?
Generating dynamic tools and using code interpreter...


2025-02-03 13:04:39,427 - MyApp - INFO - Handling tool call: execute_python_code
2025-02-03 13:04:39,429 - MyApp - INFO - Tool arguments: {'python_code': "import pandas as pd\nimport numpy as np\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.preprocessing import StandardScaler\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Load the dataset\nfile_path = '/home/sandboxuser/traffic_accidents.csv'\ndf = pd.read_csv(file_path)\n\n# Show basic information\nprint('Dataset shape:', df.shape)\nprint('First few rows:')\nprint(df.head(), '\\n')\nprint('Columns:', df.columns.tolist(), '\\n')\n\n# Correlation matrix analysis\ncorr_matrix = df.corr()\nprint('Correlation matrix:')\nprint(corr_matrix, '\\n')\n\n# Correlation of each feature with accidents\nacc_corr = corr_matrix['accidents'].drop('accidents').sort_values(key=lambda x: abs(x), ascending=False)\nprint('Correlation of other variables with accidents (sorted by absolute correlation):')\nprint(acc_corr, '\\n'

Output...
The analysis shows that one variable stands out by far:

• Both the simple correlation analysis and regression results indicate that traffic_fine_amount is the dominant factor—its correlation with accidents is strong (about –0.75), and in the standardized linear regression its coefficient is the largest in magnitude (around –3.89). The negative sign suggests that, in this data, higher fine amounts are associated with fewer accidents (which might reflect more stringent enforcement or deterrence).

Other findings include:

• The Random Forest model also ranks traffic_fine_amount as most important (importance ≈ 0.58), with the next most influential factor being traffic_density (importance ≈ 0.17). Although its simple correlation with accidents is lower, traffic_density may contribute non‐linearly.

• Additional factors like urban_area, average_speed, rain_intensity, and time_of_day have moderate associations (with linear model coefficients ranging between about ±0.5 to +0.74). T

この例では、**o3-mini**がユーザーの質問に基づいてデータを分析するためのツール（Pythonスクリプト）を動的に生成しました。**o3-mini**は相関分析、線形回帰、ランダムフォレストモデルなど、複数のアプローチを使用して問題を検証したことに注目してください。このアプローチは以下の点を強調しています：

**reasoning_effort**: モデルが実行する推論の深さ（例：この場合のアプローチ数）は、一般的にパラメータをlow、medium、highに増加させると向上します。異なるレベルの推論努力を試して、その違いを確認することができます。

**動的に生成されるツール呼び出し**: データを分析するためのツール（Pythonスクリプト）は、開発者によって手動で書かれたり、事前に決定されたりしたものではありません。代わりに、o3-miniモデルが実行時に関連するデータ探索と相関分析のコードを作成しました。

**分離されたコード実行**: セキュリティを確保し、ホストマシン上で信頼できないコードの実行を避けるため、Pythonスクリプトは`execute_python_code`ツールを使用してDockerコンテナ内で実行されました。このコンテナはリソースアクセスが制限されており（例：ネットワークなし、ファイルシステムアクセス制限）、任意のコード実行によって生じる潜在的なリスクを最小限に抑えています。

### 結論

このCookbookでは、言語の制約、コストの考慮、異なるLLMやモデルでの柔軟性の必要性など、ベンダー提供のソリューションで見られる制限に対処し、特定のアプリケーションニーズに合わせた**カスタムコードインタープリター**を開発するためのガイドを提供しています。

**エージェントとツールの管理アプローチ**: また、エージェントとツールを管理するためのコアクラスのセットを定義しました。このアプローチにより、エージェントとツールが一貫したインターフェースを持ち、異なるアプリケーション間で再利用できることが保証されます。[registry](https://github.com/openai/openai-cookbook/tree/main/examples/object_oriented_agentic_approach/resources/registry)フォルダのようなエージェントとツールのリポジトリを作成して、エージェントとツールを管理することができます。

**o3-miniモデル**: ユーザーの最小限のプロンプトに基づいてデータを分析するために、実行時に洗練されたコードを生成するo3-miniモデルの能力を実証しました。o3-miniモデルは、分析の結果を推論して、ユーザーに結果を説明しました。

最後に、**要約すると**、動的ツール呼び出しを使用したAgenticアプリケーションを構築するための3つのステップは以下の通りです：
1. 分離されたコード実行コンテナ環境のセットアップ
2. エージェントの定義とテスト
3. アプリケーションを実行するためのAgentic Orchestrationのセットアップ

セキュリティを確保し、ホストマシン上で信頼できないコードの実行を避けるために、コード実行環境を分離することの重要性について議論しました。CSVファイルのユースケースを使用して、データを分析してユーザーの質問に答えるためのツール（Pythonスクリプト）を動的に生成する方法を実証しました。また、Dockerコンテナでコードを実行し、出力をユーザーに返す方法も示しました。